In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import healpy as hp

import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db

In [2]:
# Pick a slicer
slicer = slicers.HealpixSlicer(nside=64)

# Set the database and query
dbAddress = 'sqlite:///enigma_1189_sqlite.db'
sqlconstraint = 'filter = "r" and night < 400'
opsdb = db.OpsimDatabase('sqlite:///enigma_1189_sqlite.db')
outDir = 'Stackers'
resultsDb = db.ResultsDb(outDir=outDir)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


By default, OpSim does not record the hour angle of each observation.  However, we have a stacker that will automatically add that column if it is requested.

In [3]:
metric = metrics.MeanMetric(col='HA')
bundle = metricBundles.MetricBundle(metric, slicer, sqlconstraint)
bgroup = metricBundles.MetricBundleGroup({0:bundle}, opsdb, outDir=outDir, resultsDb=resultsDb)

In [4]:
bgroup.runAll()
bgroup.plotAll()

Querying database with constraint filter = "r" and night < 400
Found 46128 visits
Running:  [0]
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Plotting.
monopole: 0.081992  dipole: lon: -75.9297, lat: 71.0826, amp: 1.80459
Plotting complete.


/Users/lynnej/anaconda/lib/python2.7/site-packages/sqlalchemy/sql/elements.py:3738: SAWarning: Textual SQL expression 'filter = "r" and night < ...' should be explicitly declared as text('filter = "r" and night < ...') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})


Now let's use a stacker to try out some different dithering options.  Rather than the standard fieldRA and fieldDec columns, we'll add columns that are slightly offset.

In [5]:
slicer = slicers.HealpixSlicer(nside=64, lonCol='nightlyRandomRADither', latCol='nightlyRandomDecDither')

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [6]:
metric = metrics.CountMetric(col='night')

In [7]:
import lsst.sims.maf.stackers as stackers
maxDither = 0.1
stackerList = [stackers.NightlyRandomDitherStacker(maxDither=maxDither)]

In [8]:
plotDict={'colorMax':50, 'xlabel':'Number of Visits', 'label':'max dither = %.2f' % maxDither}
bundle = metricBundles.MetricBundle(metric, slicer, sqlconstraint, stackerList=stackerList, plotDict=plotDict)
bgroup = metricBundles.MetricBundleGroup({0:bundle}, opsdb, outDir=outDir, resultsDb=resultsDb)

In [9]:
bgroup.runAll()
bgroup.plotAll()

Querying database with constraint filter = "r" and night < 400
Found 46128 visits
Running:  [0]
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Plotting.
monopole: 14.239  dipole: lon: 57.7042, lat: -76.6172, amp: 7.68106
Plotting complete.


Optimal bin calculation tried to make 558 bins, returning 200


In [10]:
# Update the stacker to use a larger max dither and re-run the bundle
maxDither = 1.75
plotDict2 = {'colorMax':50, 'xlabel':'Number of Visits', 'label':'max dither = %.2f' % maxDither}
stackerList = [stackers.NightlyRandomDitherStacker(maxDither=maxDither)]
bundle2 = metricBundles.MetricBundle(metric, slicer, sqlconstraint, stackerList=stackerList, plotDict=plotDict2)
bgroup2 = metricBundles.MetricBundleGroup({0:bundle2}, opsdb, outDir=outDir, resultsDb=resultsDb)


In [11]:
bgroup2.runAll()
bgroup2.plotAll()

Querying database with constraint filter = "r" and night < 400
Found 46128 visits
Running:  [0]
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Plotting.
monopole: 13.7434  dipole: lon: 61.561, lat: -76.7856, amp: 8.59758
Plotting complete.


Optimal bin calculation tried to make 660 bins, returning 200


Now we can combine the results on a single plot if we want to more easily compare them.

In [13]:
from lsst.sims.maf.plots import HealpixPowerSpectrum
fig = plt.figure(1)
plotter = HealpixPowerSpectrum()
plotter(bundle.metricValues, slicer, {'label':'max dither 0.1'}, fignum=1)
plotter(bundle2.metricValues, slicer, {'label':'max dither 1.75'}, fignum=1)
plt.legend()

monopole: 14.239  dipole: lon: 57.7042, lat: -76.6172, amp: 7.68106
monopole: 13.7434  dipole: lon: 61.561, lat: -76.7856, amp: 8.59758
